In [1]:
import transformers
model_checkpoint = "t5-small"
model_checkpoint = 'facebook/bart-base'

from datasets import load_dataset, load_metric

#raw_datasets = load_dataset("./xsum.py")
metric = load_metric("./rouge1.py")

In [2]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""
    
max_input_length = 256
max_target_length = 256

In [11]:
import numpy as np
#load npy files
vt = ['train_src', 'val_src', 'train_tar', 'val_tar']
for name in vt:
    str1 = name.split('_')[0]
    str2 = name.split('_')[1]
    str3 = str1+'_'+str2[:2]
    exec('''{} = np.load('{}.npy', allow_pickle=True)'''.format(name, str3))
train_tar_dict = {'0':[], '1':[], '2':[]}
val_tar_dict = {'0':[], '1':[], '2':[]}
train_src_dict = {'0':[], '1':[]}
val_src_dict = {'0':[], '1':[]}
for i in range(len(train_tar)):
    train_tar_dict['0'].append(train_tar[i][0])
    train_tar_dict['1'].append(train_tar[i][1])
    train_tar_dict['2'].append(train_tar[i][2])
    train_src_dict['0'].append(train_src[i][0])
    train_src_dict['1'].append(train_src[i][1])
for i in range(len(val_tar)):
    val_tar_dict['0'].append(val_tar[i][0])
    val_tar_dict['1'].append(val_tar[i][1])
    val_tar_dict['2'].append(val_tar[i][2])
    val_src_dict['0'].append(val_src[i][0])
    val_src_dict['1'].append(val_src[i][1])
train_src = train_src.tolist()
val_src = val_src.tolist()

#---------------------DEFINE LABEL-------------------------
label_list = ['O', 'B-substitute', 'I-substitute', 'B-before-insertions', 'I-before-insertions', 'B-after-insertions', 
              'I-after-insertions', 'B-revocation', 'I-revocation']
label_dict = {}
label_dict_rev = {}
for i in range(len(label_list)):
    label_dict[label_list[i]] = i
    label_dict_rev[i] = label_list[i]
short_label_list = ['O', 'substitute', 'before-insertions', 'after-insertions', 'revocation']

In [4]:
len(train_src_dict['0'][2]), len(train_src_dict['1'][2]), len(train_tar_dict['1'][2]), len(train_tar_dict['2'][2])

(22, 29, 51, 51)

In [5]:
import pandas as pd

df = pd.read_csv('1.csv', converters={'label': eval})
df = df.drop(df[df['target_new_content']=='0'].index)
df = df.reset_index(drop=True)
df_final = pd.DataFrame(data=None, columns=df.columns)

from IPython.display import display, HTML
display(HTML(df.head(5).to_html()))

,source_instruction,target_old_content,target_new_content,label,seq_len
0,"In subsections (1) and (2)(b) , for “Authority” substitute “regulators”.",comply with any requirements specified in rules made by the Authority.,comply with any requirements specified in rules made by the regulators .,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-substitute]","[10, 11, 21]"
1,"In subsections (1) and (2)(b) , for “Authority” substitute “regulators”.","At least once a year , the scheme manager must make a report to the Authority on the discharge of its functions.","At least once a year , the scheme manager must make a report to the regulators on the discharge of its functions.","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-substitute, O, O, O, O, O, O]","[10, 22, 32]"
2,"In subsections (1) and (2)(b) , for “Authority” substitute “regulators”.",comply with any requirements specified in rules made by the regulators .,comply with any requirements specified in rules made by the regulators .,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[10, 12, 22]"
3,"In subsections (1) and (2)(b) , for “Authority” substitute “regulators”.","At least once a year , the scheme manager must make a report to the regulators on the discharge of its functions.","At least once a year , the scheme manager must make a report to the regulators on the discharge of its functions.","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[10, 22, 32]"
4,"In section 322 (rules applicable to former underwriting members) , in subsection (1) , for “The Authority” substitute “The PRA” ,",The Authority may make rules imposing such requirements on persons to whom the rules apply as appear to it to be appropriate for protecting policyholders against the risk that those persons may not be able to meet their liabilities.,The PRA may make rules imposing such requirements on persons to whom the rules apply as appear to it to be appropriate for protecting policyholders against the risk that those persons may not be able to meet their liabilities.,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-substitute, I-substitute, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[21, 39, 60]"


In [7]:
from tqdm import tqdm
#--------------------------PREPROCESS DF DATA-------------------
def read_data(df, df_final):
    src_lists = []
    tar_lists = []
    type_lists = []
    set_type = {'substitute', 'insert', 'omit'}
    for i in tqdm(range(df.shape[0])):
        sr = []
        src = df.loc[i, 'source_instruction'].split(' ')
        tar = df.loc[i, 'target_old_content'].split(' ')
        tar_new = df.loc[i, 'target_new_content'].split(' ')
        masked_tar = []
        
        src = [s.strip() for s in src]
        tar = [t.strip() for t in tar]
        tar_new = [t.strip() for t in tar_new]
        length_src = len(src)
        length_tar = len(tar)
        length_tar_new = len(tar_new)
        
        label = df.loc[i, 'label']
        if set(label) == set('O') or len(src+tar)!=len(label):
            continue
        if 'the definition of' in df.loc[i, 'source_instruction']:
            continue
        #删除substitute或者insert或者omit后没有内容的
        if src[-1] in set_type:
            continue
        if 'B-revocation' in label:
            continue
        '''
        flag = 0
        for j in range(length_src):
            if flag == 1:
                sr.append(src[j]) 
            if src[j] in set_type:
                sr.append(src[j]) 
                sr.append('<type>')
                flag = 1
        '''
        
        label_list = [0]*len(src) + [label_dict.get(j, -100) for j in label[len(src):]]
        type_list = [0]*len(src) + [0 if j.split('-')[-1]=='substitute' or
                                          j.split('-')[-1]=='revocation' else 1 for j in label[len(src):]] 
        flag = 0
        for j in range(length_src):
            if flag == 1:
                type_list[j] = 1
            if src[j] in set_type:
                flag = 1
                type_list[j] = 1
            sr.append(src[j]) 
           
        for j in range(length_tar):
            if label[len(src):][j] == 'O' or label[len(src):][j] == 'I-revocation' or label[len(src):][j] == 'I-before-insertions':
                masked_tar.append(tar[j])
            if label[len(src):][j] == 'B-substitute' or label[len(src):][j] == 'B-revocation':
                masked_tar.append('<change>')
            if label[len(src):][j] == 'I-substitute':
                continue
            if label[len(src):][j] == 'B-before-insertions':
                masked_tar.append('<change>')
                masked_tar.append(tar[j])
            if j == len(tar)-1 and label[len(src):][j].split('-')[-1] == 'insertions':
                masked_tar.append(tar[j])
                masked_tar.append('<change>')
            if j != len(tar)-1:
                if label[len(src):][j] == 'B-after-insertions' and label[len(src):][j+1] == 'O':
                    masked_tar.append(tar[j])
                    masked_tar.append('<change>')
                elif label[len(src):][j] == 'I-after-insertions' and label[len(src):][j+1] == 'O':
                    masked_tar.append(tar[j])
                    masked_tar.append('<change>')
                elif label[len(src):][j].split('-')[-1] == 'insertions' and label[len(src):][j+1] != 'O':
                    masked_tar.append(tar[j])
        
        df_final = df_final.append(df.loc[i], ignore_index=True)
        src_lists.append([sr, tar])
        tar_lists.append([tar_new, type_list, label_list]) 
    return src_lists, tar_lists, df_final
src_lists, tar_lists, df_final = read_data(df, df_final)
df_final.to_csv('111.csv', index=False)
print(len(src_lists), len(tar_lists))

100%|██████████| 11853/11853 [00:28<00:00, 418.39it/s]


9717 9717


In [8]:
for i in range(2):
    #if 'omit' in src_lists[i][0]:
    print(src_lists[i])
    print(tar_lists[i])
    print(len(src_lists[i][0]+src_lists[i][1]), len(tar_lists[i][1]))
    print('---------------------------')

[['In', 'subsections', '(1)', 'and', '(2)(b)', ',', 'for', '“Authority”', 'substitute', '“regulators”.'], ['comply', 'with', 'any', 'requirements', 'specified', 'in', 'rules', 'made', 'by', 'the', 'Authority.']]
[['comply', 'with', 'any', 'requirements', 'specified', 'in', 'rules', 'made', 'by', 'the', 'regulators', '.'], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
21 21
---------------------------
[['In', 'subsections', '(1)', 'and', '(2)(b)', ',', 'for', '“Authority”', 'substitute', '“regulators”.'], ['At', 'least', 'once', 'a', 'year', ',', 'the', 'scheme', 'manager', 'must', 'make', 'a', 'report', 'to', 'the', 'Authority', 'on', 'the', 'discharge', 'of', 'its', 'functions.']]
[['At', 'least', 'once', 'a', 'year', ',', 'the', 'scheme', 'manager', 'must', 'make', 'a', 'report', 'to', 'the', 'regulators', 'on', 'the', 'discharge', 'of', 'its', 'functions.'], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [21]:
str1 = 'for “the Authority” substitute “the regulators”.'
str2 = 'in rules made by the Authority.'
list1 = ['"', 'f', 'motherfucker', 'fuck ', 'you', 'gfhh', 'fdg']
list1 = str1.split(' ')
list2 = str2.split(' ')
#list1 = [a.strip() for a in list1]
print(len(list1), len(list2))
tokenized_example = tokenizer(
                    list1,
                    list2, 
                    max_length=512,
                    truncation="only_second",
                    is_split_into_words=True,
                    return_offsets_mapping=True,
                    padding=True,
                    )
print(tokenizer.tokenize(' '.join(list2)))
print(tokenized_example["input_ids"])
om = tokenized_example["offset_mapping"]
#print(om)
label = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-substitute', 'I-substitute'] 
doc_labels = [label_dict[i] for i in label]
doc_enc_labels = np.ones(len(om),dtype=int) * -100
#print(doc_enc_labels)
arr_offset = np.array(om)
#print(arr_offset)
l = len(doc_enc_labels[(arr_offset[:,0] == 1)])
#print(l)
#doc_enc_labels[(arr_offset[:,0] == 1)] = doc_labels[:l]
word_ids = tokenized_example.word_ids(0)
length = len(list1)
cnt = 0
for i, ids in enumerate(word_ids):
    if word_ids[i] is None:
        cnt += 1
        continue
    if cnt == 3:
        word_ids[i] += length 
print(word_ids)
res = [-100 if j is None else doc_labels[j] for j in word_ids]
print(res)
res=[]
ress = ''
for j in word_ids:
    if j is None or j==ress:
        res.append(-100)
        continue
    res.append(doc_labels[j])
    ress = j 
print(res)
word_idx_dict = {}
for index, word_idx in enumerate(word_ids):
    if word_idx is None or word_idx in word_idx_dict:
        continue
    word_idx_dict[word_idx] = index
print(word_idx_dict)

6 6
['Ġin', 'Ġrules', 'Ġmade', 'Ġby', 'Ġthe', 'ĠAuthority', '.']
[0, 13, 44, 48, 627, 4305, 17, 46, 10268, 44, 48, 627, 5904, 17, 46, 4, 2, 2, 11, 1492, 156, 30, 5, 4305, 4, 2]
[None, 0, 1, 1, 1, 2, 2, 2, 3, 4, 4, 4, 5, 5, 5, 5, None, None, 6, 7, 8, 9, 10, 11, 11, None]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, -100, -100, 0, -100, -100, 0, 0, -100, -100, 0, -100, -100, -100, -100, -100, 0, 0, 0, 0, 0, 0, -100, -100]
{0: 1, 1: 2, 2: 5, 3: 8, 4: 9, 5: 12, 6: 18, 7: 19, 8: 20, 9: 21, 10: 22, 11: 23}


In [9]:
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np

train_src, val_src, train_tar, val_tar = train_test_split(src_lists, tar_lists, test_size=.2)
#save npy files
vt = ['train_src', 'val_src', 'train_tar', 'val_tar']
for name in vt:
    str1 = name.split('_')[0]
    str2 = name.split('_')[1]
    str3 = str1+'_'+str2[:2]
    exec('''{} = np.array({})'''.format(str3, name))
    exec('''np.save('{}.npy', {})'''.format(str3, str3))

#train_s = [i[0]+['<sent>']+i[1] for i in train_src]
#val_s = [i[0]+['<sent>']+i[1] for i in val_src]

/home/miniconda3/envs/pytorch_1.9/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [20]:
val_s[10], val_tar[10]

(['In',
  'subsection',
  '(5)',
  ',',
  'after',
  '“review',
  'of”',
  'insert',
  '“the',
  'Secretary',
  'of',
  'State’s',
  'powers',
  'under”.',
  '<sent>',
  'The',
  'Secretary',
  'of',
  'State',
  'must',
  'carry',
  'out',
  'a',
  'review',
  'of',
  'sections',
  '45',
  'and',
  '46',
  'and',
  'the',
  'preceding',
  'provisions',
  'of',
  'this',
  'section',
  'as',
  'soon',
  'as',
  'reasonably',
  'practicable',
  'after',
  'the',
  'end',
  'of',
  'the',
  'period',
  'of',
  '5',
  'years',
  'beginning',
  'with',
  'the',
  'day',
  'on',
  'which',
  'they',
  'come',
  'into',
  'force.'],
 array([list(['The', 'Secretary', 'of', 'State', 'must', 'carry', 'out', 'a', 'review', 'of', 'the', 'Secretary', 'of', "State's", 'powers', 'under', 'sections', '45', 'and', '46', 'and', 'the', 'preceding', 'provisions', 'of', 'this', 'section', 'as', 'soon', 'as', 'reasonably', 'practicable', 'after', 'the', 'end', 'of', 'the', 'period', 'of', '5', 'years', 'be

In [4]:
SPECIAL_TOKEN = {'additional_special_tokens': ['<change>', '<sent>', '<type>']}
SPECIAL_TOKEN = {'additional_special_tokens': ['<sent>']}
tokenizer.add_special_tokens(SPECIAL_TOKEN)

1

In [12]:
def get_entity_bio(seq, id2label):
        """Gets entities from sequence.
        note: BIO
        Args:
            seq (list): sequence of labels.
        Returns:
            list: list of (chunk_type, chunk_start, chunk_end).
        Example:
            seq = ['B-PER', 'I-PER', 'O', 'B-LOC']
            get_entity_bio(seq)
            #output
            [['PER', 0,1], ['LOC', 3, 3]]
        """
        chunks = []
        chunk = [-1, -1, -1]
        for indx, tag in enumerate(seq):
            if not isinstance(tag, str):
                tag = id2label[tag]
            if tag.startswith("B-"):
                if chunk[2] != -1:
                    chunks.append(chunk)
                chunk = [-1, -1, -1]
                chunk[1] = indx
                chunk[0] = '-'.join(tag.split('-')[1:])
                chunk[2] = indx
                if indx == len(seq) - 1:
                    chunks.append(chunk)
            elif tag.startswith('I-') and chunk[1] != -1:
                _type = '-'.join(tag.split('-')[1:])
                if _type == chunk[0]:
                    chunk[2] = indx

                if indx == len(seq) - 1:
                    chunks.append(chunk)
            else:
                if chunk[2] != -1:
                    chunks.append(chunk)
                chunk = [-1, -1, -1]
        return chunks
    
def preprocess_function(src, tar, ner_label, mask):
    short_label_list = ['O', 'substitute', 'before-insertions', 'after-insertions', 'revocation']
    label2id = {label: i for i, label in enumerate(short_label_list)}
    attention_mask = []
    start_ids_labels = []
    end_ids_labels = []
    '''
    model_inputs = tokenizer(src['0'],
                             src['1'],
                             is_split_into_words=True,
                             max_length=max_input_length, 
                             truncation='only_second',
                             padding=True,)
    '''
    model_inputs = tokenizer(src['0'],
                             src['1'],
                             is_split_into_words=True,
                             max_length=max_input_length, 
                             truncation='only_second',
                             padding=True,)
    for i in range(len(mask)):
        
        length = len(src['0'][i])
        word_ids = model_inputs.word_ids(i)
        cnt = 0
        for j, ids in enumerate(word_ids):
            if word_ids[j] is None:
                cnt += 1
                continue
            if cnt == 3:
                word_ids[j] += length 
        word_idx_dict = {}
        for n in range(len(word_ids)):
            if word_ids[n] in word_idx_dict or word_ids[n] is None:
                continue
            word_idx_dict[word_ids[n]] = n
        
        m = mask[i]
        token_mask = [0] * len(word_ids)
        #print(i, word_ids, word_idx_dict, m, len(word_ids), len(m))
        for j in range(len(word_ids)):
            if word_ids[j] is None:
                continue
            else:
                token_mask[j] = m[word_ids[j]]
        #ner_labels.append()
        attention_mask.append(token_mask)
            
        start_ids = [0] * len(word_ids)
        end_ids = [0] * len(word_ids)
        n = 0
        subjects = get_entity_bio(ner_label[i], label_dict_rev)
        for subject in subjects:
            label = subject[0]
            start = subject[1]
            end = subject[2]
            #print(start, end)
            if start in word_idx_dict and end in word_idx_dict:
                start_ids[word_idx_dict[start]] = label2id[label]
                end_ids[word_idx_dict[end]] = label2id[label]
                n+=3
        start_ids_labels.append(start_ids)
        end_ids_labels.append(end_ids)
        
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tar, 
                           is_split_into_words=True, 
                           max_length=max_target_length, 
                           truncation=True)
    #model_inputs['attention_mask'] = attention_mask
    model_inputs['start_positions'] = start_ids_labels
    model_inputs['end_positions'] = end_ids_labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
train_encodings = preprocess_function(train_src_dict, train_tar_dict['0'], train_tar_dict['2'], train_tar_dict['1'])
val_encodings = preprocess_function(val_src_dict, val_tar_dict['0'], val_tar_dict['2'], val_tar_dict['1'])

In [17]:
val_encodings[0]

Encoding(num_tokens=256, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [14]:
import torch

class Seq2seqdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, type_id_lists):
        self.encodings = encodings
        self.type_id_lists = type_id_lists

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #item['type_ids'] = torch.tensor(self.type_id_lists[idx])
        item['labels'] = torch.tensor(self.encodings['labels'][idx])
        return item

    def __len__(self):
        return len(self.encodings['labels'])

train_dataset = Seq2seqdataset(train_encodings, train_tar_dict['1'])
val_dataset = Seq2seqdataset(val_encodings, val_tar_dict['1'])

In [15]:
val_dataset[0]

{'input_ids': tensor([    0,    96, 45845,    36,   134,    43,  2156,    13,    44,    48,
         45580,    17,    46, 10268,    44,    48,   771, 44870,  3389,  1621,
            50,     5,   496,  3389,    13,  5295,  1463,    17,    46,     4,
             2,     2,   660,   645,   223,  2810,   971,  1640,   176,    43,
           189,    45, 43388,     5,  3389,    31, 22557,   155,     4,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [8]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartConfig
from models import BartNerJointModel
import nltk
import numpy as np

num_labels = len(short_label_list)
config = BartConfig.from_pretrained(model_checkpoint,num_labels=num_labels)
config.loss_type = 'ce'
model = BartNerJointModel.from_pretrained(model_checkpoint, config=config)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
#model.resize_token_embeddings(len(tokenizer))

Some weights of BartNerJointModel were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['dense1.bias', 'dense1.weight', 'dense2.bias', 'dense2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_strategy ='epoch',
    num_train_epochs=5,
    load_best_model_at_end=True,
    predict_with_generate=True,
    generation_num_beams=3,
    generation_max_length=512,
    #fp16=True,
    push_to_hub=False,
    no_cuda=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    cnt = 0
    for i in range(len(decoded_preds)):
        if decoded_preds[i] == decoded_labels[i]:
            cnt += 1
    ratio = cnt/len(decoded_preds)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["Strict-match-ratio"] = ratio
    return {k: round(v, 4) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

***** Running training *****
  Num examples = 7793
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4875


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Strict-match-ratio
1,0.248800,0.121602,95.859900,94.557500,95.652300,95.622500,45.295000,0.551100
2,0.112800,0.104435,95.976000,94.782900,95.760300,95.757400,44.989200,0.567000
3,0.079600,0.099790,96.105000,94.875500,95.858700,95.860400,45.532100,0.571600
4,0.063000,0.096284,95.922800,94.759300,95.751400,95.742600,44.654700,0.578800


***** Running Evaluation *****
  Num examples = 1949
  Batch size = 8
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-975
Configuration saved in bart-base-finetuned-xsum/checkpoint-975/config.json
Model weights saved in bart-base-finetuned-xsum/checkpoint-975/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum/checkpoint-975/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum/checkpoint-975/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1949
  Batch size = 8
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-1950
Configuration saved in bart-base-finetuned-xsum/checkpoint-1950/config.json
Model weights saved in bart-base-finetuned-xsum/checkpoint-1950/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum/checkpoint-1950/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum/checkpoint-1950/special_tokens_map.json
***** Running Evaluation *****
  Num 

KeyboardInterrupt: 

In [12]:
t1 = torch.tensor([1,21,3])
t2 = torch.tensor([2,32,3])
t3 = torch.stack((t1, t2, t2, t2))
t4 = (t1, t2)
torch.stack(t4)

tensor([[ 1, 21,  3],
        [ 2, 32,  3]])

In [12]:
from transformers.modeling_outputs import Seq2SeqLMOutput

In [ ]:
#model = AutoModelForSeq2SeqLM.from_pretrained('./bart-base-finetuned-xsum/checkpoint-7794')
#model.to('cuda')
#model.eval()

trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer)

def predict_report(trainer, dataset):
    
    predictions, label_ids, metrics = trainer.predict(dataset, num_beams=3, max_length=512)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    print(len(decoded_preds[0]), len(decoded_preds[1]))
    print(len(decoded_preds))
    
    cnt = 0
    for i in range(200):
        if decoded_preds[i] == decoded_labels[i]:
            cnt += 1
            print('Same:', decoded_preds[i])
        else:
            print('Diff:', decoded_preds[i])
            print('.......')
            print(decoded_labels[i])
        print('----------------------------------')
    ratio = cnt/len(decoded_preds)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}, {'ratio':ratio}

print(predict_report(trainer, val_dataset))

In [27]:
a = val_s[10] 
b = val_tar[10]
c = [a[i] for i in range(len(a)) if b[1][i]==1]
print(b[0], c)

['a', 'member', 'of', 'a', 'police', 'force', 'seconded', 'to', 'the', 'National', 'Crime', 'Agency', 'to', 'serve', 'as', 'a', 'National', 'Crime', 'Agency', 'officer', 'is', 'to', 'be', 'treated', 'as', 'employed', 'by', 'that', 'Agency.'] ['substitute', '“seconded', 'to', 'the', 'National', 'Crime', 'Agency', 'to', 'serve', 'as', 'a', 'National', 'Crime', 'Agency', 'officer', 'is', 'to', 'be', 'treated”.', '<sent>', 'a', 'member', 'of', 'a', 'police', 'force', 'as', 'employed', 'by', 'that', 'Agency.']


In [18]:
from rouge import Rouge 

src = 'for the words “Supreme Court Act 1981”, substitute “ , that the decision”'
tar_before = 'after paragraph (e) of <change> 1981 insert—“(ea) proceedings under section 79 of the Childcare Act 2006;”'
tar_after = 'after paragraph (e) of , that the decision insert—“(ea) proceedings under section 79 of the Childcare Act 2006;”'
'''
src = 'for the words from “a condition of a direction” to “specified by him”, substitute “fuck you”'
tar_before ='The Secretary of State may make it <change>.'
tar_after = 'The Secretary of State may make it fuck you.'

src = 'in paragraph (2), for “23(3) or 24(3)” substitute “23(1)(de)(ii), (1)(df)(ii), (3), 24(1)(ce)(ii), (1)(cf)(ii) or (3)”.'
tar_before = 'Where a person has his appointment revoked or if an approval given in respect of him under regulation <change> is withdrawn, that person shall immediately return to the Secretary of State all forms of pass certificates supplied to him under regulations 47(8) and 48(3) which he still holds.'
tar_after = 'Where a person has his appointment revoked or if an approval given in respect of him under regulation 23(1)(de)(ii), (1)(df)(ii), (3), 24(1)(ce)(ii), (1)(cf)(ii) or (3) is withdrawn, that person shall immediately return to the Secretary of State all forms of pass certificates supplied to him under regulations 47(8) and 48(3) which he still holds.'

src = 'In section 86 of the Act (exempt offers to the public) , in subsections (1A)(c) and (1B)(a) , after “the FCA” insert “or the competent authority of another EEA State”.'
tar_before = 'a prospectus is available for the securities which has been approved by the FCA <change> and meets either of the conditions in subsection (1B) ; and'
tar_after = 'a prospectus is available for the securities which has been approved by the FCA or the competent authority of another EEA State and meets either of the conditions in subsection (1B) ; and'

src = 'In subsection (5EA), omit "and the reference to the Registrar General in subsection (5C) accordingly has effect as a reference to the Secretary of State".'
tar_before = 'If a proposed marriage is referred to the Secretary of State under section 28H— (a)any application under subsection (5A) is to be made to the Secretary of State; and (b)the power conferred by subsection (5A) is exercisable by the Secretary of State <change>.'
tar_after = 'If a proposed marriage is referred to the Secretary of State under section 28H— (a)any application under subsection (5A) is to be made to the Secretary of State; and (b)the power conferred by subsection (5A) is exercisable by the Secretary of State ... .'

src = 'for "the superintendent registrar" (in the first place it appears) substitute "any superintendent registrar";'
tar_before = 'The provisions of section 29 of the principal Act (caveat against issue of certificate) shall apply to the issue of a licence by the Registrar General with the modification that a caveat may be entered with either <change> or the Registrar General and in either case it shall be for the Registrar General to examine into the matter of the caveat and to decide whether or not the licence should be granted and his decision shall be final, and with a further modification that the references to the superintendent registrar in that section shall refer to the superintendent registrar of the registration district in which the marriage is intended to be solemnised.'
tar_after = 'The provisions of section 29 of the principal Act (caveat against issue of certificate) shall apply to the issue of a licence by the Registrar General with the modification that a caveat may be entered with either any superintendent registrar or the Registrar General and in either case it shall be for the Registrar General to examine into the matter of the caveat and to decide whether or not the licence should be granted and his decision shall be final, and with a further modification that the references to the superintendent registrar in that section shall refer to the superintendent registrar of the registration district in which the marriage is intended to be solemnised.'
'''
set_type = {'substitute', 'insert', 'omit'}
src_list = src.split(' ')
src = []
'''
flag = 0
for i in src_list:
    if flag == 1:
        src.append(i)
    if i in set_type:
        src.append(i)
        src.append('<type>') 
        flag = 1
'''
for i in src_list:
    src.append(i)
    #if i in set_type:
        #src.append('<type>') 
        
input_list = src + ['<sent>'] + tar_before.split(' ')
print(input_list)
test_encodings = tokenizer(input_list, is_split_into_words=True,
                           max_length=max_input_length,
                           return_offsets_mapping=True, truncation=True)
model.eval()
summary_ids = model.generate(torch.tensor([test_encodings["input_ids"]]).cuda(), num_beams=3, max_length=512, early_stopping=True)
hypothesis = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

hypothesis = hypothesis[0].strip()
reference = tar_after

print(hypothesis, '\n', reference)
print(hypothesis==reference)
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
scores

['for', 'the', 'words', '“Supreme', 'Court', 'Act', '1981”,', 'substitute', '“', ',', 'that', 'the', 'decision”', '<sent>', 'after', 'paragraph', '(e)', 'of', '<change>', '1981', 'insert—“(ea)', 'proceedings', 'under', 'section', '79', 'of', 'the', 'Childcare', 'Act', '2006;”']
after paragraph (e) of , that the decision 1981 insert —“(ea) proceedings under section 79 of the Childcare Act 2006;” 
 after paragraph (e) of , that the decision insert—“(ea) proceedings under section 79 of the Childcare Act 2006;”
False


[{'rouge-1': {'r': 0.9375, 'p': 0.8333333333333334, 'f': 0.8823529361937716},
  'rouge-2': {'r': 0.8823529411764706,
   'p': 0.7894736842105263,
   'f': 0.8333333283487654},
  'rouge-l': {'r': 0.9375, 'p': 0.8333333333333334, 'f': 0.8823529361937716}}]

In [51]:
result = metric.compute(predictions=['a a a'], references=['a a a'], use_stemmer=True)
result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
result

{'rouge1': 100.0, 'rouge2': 100.0, 'rougeL': 100.0, 'rougeLsum': 100.0}

In [ ]:
import os

import torch
import torch.nn as nn
from transformers import AutoModelForSeq2SeqLM
from models.seq2seq import *

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# model = AutoModelForSeq2SeqLM.from_pretrained('./weights/annotation_gen_BART')
# tokenizer = AutoTokenizer.from_pretrained('./weights/annotation_gen_BART')
#
# ARTICLE_TO_SUMMARIZE = "In Article 8 (civil-military coordination), in each of paragraphs 1 to 3 for “Member States” substitute “The Secretary of State”."
# inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
#
# # Generate Summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=30, early_stopping=True)
# [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained('./weights/annotation_gen_BART/')

    def forward(self, input_ids):
        return self.model.generate(input_ids, num_beams=4, max_length=30, early_stopping=True)


def export_seq2seq_onnx_representation(model_path='./bart-base-finetuned-xsum/checkpoint-1220', save_path='./'):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    model.eval()
    encoder = model.model.encoder
    decoder = model.model.decoder
    lm_head = model.lm_head

    decoder_with_lm_head = CombinedDecoder(decoder, lm_head, model.config)
    simplified_encoder = SimplifiedT5Encoder(encoder)
    # Example sequence
    input_ids = torch.tensor([[42] * 10])

    # Exports to ONNX
    _ = torch.onnx.export(
        decoder_with_lm_head.eval(),
        (input_ids, simplified_encoder(input_ids)),
        f"{save_path}/decoder.onnx",
        export_params=True,
        opset_version=12,
        input_names=['input_ids', 'encoder_hidden_states'],
        output_names=['hidden_states'],
        dynamic_axes={
            'input_ids': {0: 'batch', 1: 'sequence'},
            'encoder_hidden_states': {0: 'batch', 1: 'sequence'},
            'hidden_states': {0: 'batch', 1: 'sequence'},
        })

    _ = torch.onnx._export(
        simplified_encoder.eval(),
        input_ids,
        f"{save_path}/encoder.onnx",
        export_params=True,
        opset_version=12,
        input_names=['input_ids'],
        output_names=['hidden_states'],
        dynamic_axes={
            'input_ids': {0: 'batch', 1: 'sequence'},
            'hidden_states': {0: 'batch', 1: 'sequence'},
        }
    )


# import onnxruntime
# ort_session = onnxruntime.InferenceSession("./weights/bart_onnx/bart.onnx")
#
# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
#
# # compute ONNX Runtime output prediction
# x = torch.randint(high=100, size=(1, 256))
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
# ort_outs = ort_session.run(None, ort_inputs)
if __name__ == '__main__':
    export_seq2seq_onnx_representation()
    print("Model exported at ", './weights/bart_onnx')

In [19]:
# Some standard imports
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

In [20]:
# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# set the model to inference mode
torch_model.eval()

Downloading: "https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth" to /root/.cache/torch/hub/checkpoints/superres_epoch100-44c6958e.pth


  0%|          | 0.00/234k [00:00<?, ?B/s]

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

In [22]:
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [23]:
import onnx

onnx_model = onnx.load("super_resolution.onnx")
onnx.checker.check_model(onnx_model)

In [24]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("super_resolution.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!
